# Flux Balance Analysis using COBRApy

Simulations using flux balance analysis can be solved using COBRApy model.optimize() command. This module will maximize or minimize (maximizing is the default) flux based on the objective reactions.

To start with, import the *E.coli* core model from the BIGG database.

In [1]:
import cobra.test
from cobrapy_bigg_client import client
model = client.download_model('e_coli_core', save=False) # Load model from BIGG database
model.solver = 'glpk'

Academic license - for non-commercial use only - expires 2022-12-07
Using license file C:\Users\scott\gurobi.lic


## Running a FBA simulation

Place the results of the simulation in an object called "solution" and then print the result of the simulation which is the numerical value of the objective function which in the case corresponds to the growth rate

In [2]:
solution = model.optimize()
print(solution)

<Solution 0.874 at 0x129468075e0>


The **model.optimize()** function will return a Solution object. A solution object has several attributes:

- objective_value: the objective value
- status: the status from the linear programming solver
- fluxes: a pandas series with flux indexed by reaction identifier. The flux for a reaction variable is the difference of the primal values for the forward and reverse reaction variables.
- shadow_prices: a pandas series with shadow price indexed by the metabolite identifier.

For example, after the last call to model.optimize(), if the optimization succeeds it's status will be optimal. In case the model is infeasible an error is raised.

The objective value can also be found through either

In [3]:
solution.objective_value

0.8739215069684303

or

In [4]:
model.optimize().objective_value

0.8739215069684303

The solvers that can be used with COBRApy are so fast that for many small to mid-size models computing the solution can be even faster than it takes to collect the values from the solver and convert to them python objects. With model.optimize, COBRApy gathers values for all reactions and metabolites and that can take a significant amount of time if done repeatedly. If only the flux value of a single reaction or the objective, it is faster to use **model.slim_optimize()** which only does the optimization and returns the objective value leaving it up to you to fetch other values that you may need.

This can be seen below in the following examples

In [5]:
%%time
model.optimize().objective_value

Wall time: 2.01 ms


0.8739215069684303

In [6]:
%%time
model.slim_optimize()

Wall time: 0 ns


0.8739215069684303

## Analyzing FBA solutions
Models solved using FBA can be further analyzed by using summary methods, which produce printed text that gives a quick representation of optimized model behavior. Calling the summary method on the entire model produces information on the input and output behavior of the model, along with the optimized objective.

In [7]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


As can be seen, this summary produces the optimized objective plus all the flux values of all the exchange reactions that are actively uptaking flux into the cell as well as the flux values of the exhange reactions that are secreting flux into the  extracellular space.

### Individual Reaction Flux Values

The flux though a given reaction can be determined as follows.

In [8]:
model.reactions.PYK.flux # This does not work with the Gurobi solver

1.7581774441067866

or

In [9]:
solution.fluxes.PYK

1.7581774441068045

The reduced cost of a given reaction is given by

In [10]:
from cobra import Solution
model.reactions.PYK.reduced_cost # This does not work with the gurobi solver

-1.9999999999999991

Printing all the flux values

In [11]:
import pandas as pd
pd.set_option('display.max_rows', 500)
solution.fluxes

PFK                          7.477382
PFL                          0.000000
PGI                          4.860861
PGK                        -16.023526
PGL                          4.959985
ACALD                        0.000000
AKGt2r                       0.000000
PGM                        -14.716140
PIt2r                        3.214895
ALCD2x                       0.000000
ACALDt                       0.000000
ACKr                         0.000000
PPC                          2.504309
ACONTa                       6.007250
ACONTb                       6.007250
ATPM                         8.390000
PPCK                         0.000000
ACt2r                        0.000000
PPS                          0.000000
ADK1                         0.000000
AKGDH                        5.064376
ATPS4r                      45.514010
PTAr                         0.000000
PYK                          1.758177
BIOMASS_Ecoli_core_w_GAM     0.873922
PYRt2                        0.000000
CO2t        

Printing all the reduced costs

In [12]:
solution.reduced_costs

PFK                        -1.257675e-17
PFL                         2.081668e-17
PGI                         0.000000e+00
PGK                         6.938894e-18
PGL                         1.528725e-17
ACALD                      -1.431147e-17
AKGt2r                      8.673617e-18
PGM                        -0.000000e+00
PIt2r                      -7.806256e-18
ALCD2x                     -2.125036e-17
ACALDt                      0.000000e+00
ACKr                        0.000000e+00
PPC                         2.222614e-17
ACONTa                      1.376937e-17
ACONTb                      1.084202e-19
ATPM                       -1.018497e-02
PPCK                       -1.018497e-02
ACt2r                      -1.734723e-18
PPS                        -1.018497e-02
ADK1                       -0.000000e+00
AKGDH                      -1.734723e-18
ATPS4r                      1.192622e-18
PTAr                        1.647987e-17
PYK                         5.637851e-18
BIOMASS_Ecoli_co

Creating a table that includes the reaction IDs, reaction names, reaction flux, and reduced costs.

In [13]:
import pandas as pd
reaction_names = [r.name for r in model.reactions]
reaction_ids = [r.id for r in model.reactions]
reaction_flux = [r.flux for r in model.reactions]
reaction_RC = [r.reduced_cost for r in model.reactions]
reactionList = {'Reaction ID': reaction_ids,
                'Reaction Name': reaction_names,
                'Reaction Flux': reaction_flux,
                'Reaction Reduced Cost': reaction_RC,
               }

df = pd.DataFrame(reactionList, columns= ['Reaction ID','Reaction Name','Reaction Flux','Reaction Reduced Cost'])
pd.set_option('display.max_rows', 500)
df

,Reaction ID,Reaction Name,Reaction Flux,Reaction Reduced Cost
0,PFK,Phosphofructokinase,7.477382e+00,-2.000000
1,PFL,Pyruvate formate lyase,0.000000e+00,-2.000000
2,PGI,Glucose-6-phosphate isomerase,4.860861e+00,-2.000000
3,PGK,Phosphoglycerate kinase,-1.602353e+01,2.000000
4,PGL,6-phosphogluconolactonase,4.959985e+00,-2.000000
5,ACALD,Acetaldehyde dehydrogenase (acetylating),0.000000e+00,-2.000000
6,AKGt2r,2 oxoglutarate reversible transport via symport,0.000000e+00,-2.000000
7,PGM,Phosphoglycerate mutase,-1.471614e+01,2.000000
8,PIt2r,Phosphate reversible transport via symport,3.214895e+00,-2.000000
9,ALCD2x,Alcohol dehydrogenase (ethanol),0.000000e+00,-2.000000


### Individual Metabolite Flux Values

In addition, the input-output behavior of individual metabolites can also be inspected using summary methods. For example, the following commands can be used to examine the flux associated with the reactions that are both producing and consuming the given metabolite.

In [14]:
model.metabolites.nadh_c.summary()

Percent,Flux,Reaction,Definition
13.14%,5.064,AKGDH,akg_c + coa_c + nad_c --> co2_c + nadh_c + succoa_c
8.04%,3.1,BIOMASS_Ecoli_core_w_GAM,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
41.58%,16.02,GAPD,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
13.14%,5.064,MDH,mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
24.09%,9.283,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
Percent,Flux,Reaction,Definition
100.00%,-38.53,NADH16,4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q8h2_c


This will be helpful in determining the main energy production and consumption reactions

In [15]:
model.metabolites.atp_c.summary()

Percent,Flux,Reaction,Definition
66.58%,45.51,ATPS4r,adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
23.44%,16.02,PGK,3pg_c + atp_c <=> 13dpg_c + adp_c
2.57%,1.758,PYK,adp_c + h_c + pep_c --> atp_c + pyr_c
7.41%,5.064,SUCOAS,atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
Percent,Flux,Reaction,Definition
12.27%,-8.39,ATPM,atp_c + h2o_c --> adp_c + h_c + pi_c
76.46%,-52.27,BIOMASS_Ecoli_core_w_GAM,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c + 0.5191 pep_c + 2.8328 pyr_c + 0.8977 r5p_c --> 59.81 adp_c + 4.1182 akg_c + 3.7478 coa_c + 59.81 h_c + 3.547 nadh_c + 13.0279 nadp_c + 59.81 pi_c
0.33%,-0.2235,GLNS,atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c + h_c + pi_c
10.94%,-7.477,PFK,atp_c + f6p_c --> adp_c + fdp_c + h_c


In [16]:
model.metabolites.atp_c

Metabolite identifier,atp_c
Name,ATP C10H12N5O13P3
Memory address,0x01294543e1c0
Formula,C10H12N5O13P3
Compartment,c
In 13 reaction(s),"GLNS, ATPM, ACKr, PYK, GLNabc, ADK1, BIOMASS_Ecoli_core_w_GAM, PPS, PPCK, PFK, PGK, SUCOAS, ATPS4r"


Note that not all of the thirteen reactions associated with atp_c are carrying flux in this optimization. 

The shadow prices for each metabolite can be determined through the following command

In [17]:
model.metabolites.nadh_c.shadow_price

3.866666666666667

LIsting all the metabolite shadow prices

In [18]:
solution.shadow_prices

glc__D_e    -9.166475e-02
gln__L_c    -7.511417e-02
gln__L_e    -7.002168e-02
glu__L_c    -7.002168e-02
glu__L_e    -6.874856e-02
glx_c       -1.909682e-02
h2o_c        1.084202e-19
h2o_e        1.084202e-19
h_c          1.273121e-03
h_e          4.336809e-19
icit_c      -7.129480e-02
lac__D_c    -4.201301e-02
lac__D_e    -4.073989e-02
mal__L_c    -4.837862e-02
mal__L_e    -4.583237e-02
nad_c        7.638729e-03
nadh_c       0.000000e+00
nadp_c       8.911850e-03
nadph_c      0.000000e+00
nh4_c       -0.000000e+00
13dpg_c     -4.710549e-02
nh4_e       -1.387779e-17
o2_c        -0.000000e+00
2pg_c       -4.201301e-02
o2_e        -0.000000e+00
3pg_c       -4.201301e-02
oaa_c       -4.201301e-02
pep_c       -4.201301e-02
6pgc_c      -9.166475e-02
pi_c        -1.273121e-03
6pgl_c      -9.039162e-02
pi_e        -4.163336e-17
ac_c        -2.418931e-02
pyr_c       -3.564740e-02
pyr_e       -3.437428e-02
q8_c         2.546243e-03
q8h2_c       0.000000e+00
r5p_c       -8.275290e-02
ru5p__D_c   

Creating a table that includes the shadow prices of all the metabolites

In [19]:
metabolite_names = [m.name for m in model.metabolites]
metabolite_ids = [m.id for m in model.metabolites]
metabolite_compartment = [m.compartment for m in model.metabolites]
metabolite_formula = [m.formula for m in model.metabolites]
metabolite_charge = [m.charge for m in model.metabolites]
metabolite_SP = [m.shadow_price for m in model.metabolites]
metaboliteList = {'Metabolite ID': metabolite_ids,
                  'Metabolite Name': metabolite_names,
                  'Metabolite Formula': metabolite_formula,
                  'Metabolite Compartment': metabolite_compartment,
                  'Metabolite Charge': metabolite_charge,
                  'Metabolite Shadow Price': metabolite_SP
                 }

df = pd.DataFrame(metaboliteList, columns= ['Metabolite ID','Metabolite Name', 'Metabolite Formula', 'Metabolite Compartment','Metabolite Charge','Metabolite Shadow Price'])
pd.set_option('display.max_rows', 500)
df

,Metabolite ID,Metabolite Name,Metabolite Formula,Metabolite Compartment,Metabolite Charge,Metabolite Shadow Price
0,glc__D_e,D-Glucose,C6H12O6,e,0,5.200000
1,gln__L_c,L-Glutamine,C5H10N2O3,c,0,34.155556
2,gln__L_e,L-Glutamine,C5H10N2O3,e,0,26.911111
3,glu__L_c,L-Glutamate,C5H8NO4,c,-1,22.911111
4,glu__L_e,L-Glutamate,C5H8NO4,e,-1,22.600000
5,glx_c,Glyoxylate,C2H1O3,c,-1,8.377778
6,h2o_c,H2O H2O,H2O,c,0,-2.000000
7,h2o_e,H2O H2O,H2O,e,0,-1.000000
8,h_c,H+,H,c,1,-2.311111
9,h_e,H+,H,e,1,-1.000000


## Changing the Objective Function
The objective function is determined from the objective_coefficient attribute of the objective reaction(s). For most cases, a “biomass” function which describes the composition of metabolites which make up a cell is used. The objective function can be change to a single or multiple reactions.

The objective function for a given model can be determined using the **model.summary()** or the **linear_reaction_coefficients(model)** commands.

In [20]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc__D_e,10,6,100.00%
nh4_e,EX_nh4_e,4.765,0,0.00%
o2_e,EX_o2_e,21.8,0,0.00%
pi_e,EX_pi_e,3.215,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
co2_e,EX_co2_e,-22.81,1,100.00%
h2o_e,EX_h2o_e,-29.18,0,0.00%
h_e,EX_h_e,-17.53,0,0.00%


or

In [21]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction BIOMASS_Ecoli_core_w_GAM at 0x1294548f6d0>: 1.0}

The details of the objective function can be found by using the following code(the default objective function for the *E.coli* core model is the biomass function)

In [22]:
biomass_rxn = model.reactions.get_by_id("BIOMASS_Ecoli_core_w_GAM")
biomass_rxn

Reaction identifier,BIOMASS_Ecoli_core_w_GAM
Name,Biomass Objective Function with GAM
Memory address,0x01294548f6d0
Stoichiometry,1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0.361 e4p_c + 0.0709 f6p_c + 0.129 g3p_c + 0.205 g6p_c + 0.2557 gln__L_c + 4.9414 glu__L_c + 59.81 h2o_c + 3.547 nad_c + 13.0279 nadph_c + 1.7867 oaa_c... 1.496 3-Phospho-D-glycerate + 3.7478 Acetyl-CoA + 59.81 ATP C10H12N5O13P3 + 0.361 D-Erythrose 4-phosphate + 0.0709 D-Fructose 6-phosphate + 0.129 Glyceraldehyde 3-phosphate + 0.205 D-Glucose...
GPR,
Lower bound,0.0
Upper bound,1000.0


Currently in the model, there is only one reaction in the objective (the biomass reaction), with an linear coefficient of 1.

The objective function can be changed by assigning model.objective, which can be a reaction object (or just it’s name), or a dict of {Reaction: objective_coefficient}.

In [23]:
# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.

Verifying this change to the objective function

In [24]:
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x12945471430>: 1.0}

The characteristics of the new objective function

In [25]:
objectiveFunction = model.reactions.get_by_id("ATPM")
objectiveFunction

Reaction identifier,ATPM
Name,ATP maintenance requirement
Memory address,0x012945471430
Stoichiometry,atp_c + h2o_c --> adp_c + h_c + pi_c ATP C10H12N5O13P3 + H2O H2O --> ADP C10H12N5O10P2 + H+ + Phosphate
GPR,
Lower bound,8.39
Upper bound,1000.0


The numerical value of the new ATPM objective function is given below. Notice that the model is no longer optimizing for cell growth but now it is maximizing the production of ATP.

In [26]:
model.optimize().objective_value

175.0000000000001

## Running Geometric FBA
Geometric FBA finds a unique optimal flux distribution which is central to the range of possible fluxes. For more details on geometric FBA, please see K Smallbone, E Simeonidis (2009).

In [27]:
model.objective = "BIOMASS_Ecoli_core_w_GAM"
geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
geometric_fba_sol

,fluxes,reduced_costs
PFK,7.477382,0.0
PFL,0.000000,0.0
PGI,4.860861,0.0
PGK,-16.023526,0.0
PGL,4.959985,0.0
...,...,...
NADH16,38.534610,0.0
NADTRHD,0.000000,0.0
NH4t,4.765319,0.0
O2t,21.799493,0.0


## Plotting the Fluxes on an Escher Map

In [28]:
import cobra
import escher
from escher import Builder

To plot the maps of the different models requires a Jupyter widget called "Builder." This widget will execute in a Jupyter cell to embed an Escher map. Two names are required for the widget to operate properly; 1) the "map_name" is the name of the map file (different from the model file) that contains all the information to plot a map of the organism, 2) the "model_name" is the name of a genome-scale metabolic network reconstruction or COBRA model. The maps and models that are supported by the Escher website include the following.

In [29]:
escher.list_available_maps()

[{'organism': 'Saccharomyces cerevisiae',
  'map_name': 'iMM904.Central carbon metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Inositol retinol metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Glycolysis TCA PPP'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Tryptophan metabolism'},
 {'organism': 'Homo sapiens', 'map_name': 'RECON1.Carbohydrate metabolism'},
 {'organism': 'Homo sapiens',
  'map_name': 'RECON1.Amino acid metabolism (partial)'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Nucleotide metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid biosynthesis (saturated)'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Nucleotide and histidine biosynthesis'},
 {'organism': 'Escherichia coli', 'map_name': 'e_coli_core.Core metabolism'},
 {'organism': 'Escherichia coli', 'map_name': 'iJO1366.Central metabolism'},
 {'organism': 'Escherichia coli',
  'map_name': 'iJO1366.Fatty acid beta-oxidation'}

In [30]:
escher.list_available_models()

[{'organism': 'Saccharomyces cerevisiae', 'model_name': 'iMM904'},
 {'organism': 'Homo sapiens', 'model_name': 'RECON1'},
 {'organism': 'Escherichia coli', 'model_name': 'e_coli_core'},
 {'organism': 'Escherichia coli', 'model_name': 'iJO1366'}]

To load a map and model into the Builder widget can be done as follows

In [31]:
builder = Builder(
    map_name='e_coli_core.Core metabolism', 
    model_name='e_coli_core', # This loads the model name into Escher but does not change the model in the COBRApy simulation
)

This sets the model and map name for the Escher package. The model name that is downloaded does not replace the model used in the COBRApy simulations.

In [32]:
builder

Builder()

Note that the flux data is overlayed on the map that was created by the Builder widget. Plotting the flux data from an FBA simulation (model.optimize()) onto the map. If you zoom in on the map and click the arrow to the left you can put the arrow over a reaction or metabolite and see the flux value(reactions). 

In [33]:
solution = model.optimize()
builder.reaction_data = solution.fluxes

Now lets create a new map that also includes the shadow prices of the metabolites.

In [34]:
builder = Builder(
    map_name='e_coli_core.Core metabolism', 
    model_name='e_coli_core', 
)
builder

Builder()

If you zoom in on the map and click the arrow to the left you can move the arrow over a metabolite's name and the shadow price information for the metabolite will be shown. 

In [35]:
solution = model.optimize()
builder.reaction_data = solution.fluxes
builder.metabolite_data = solution.shadow_prices

More information on how to use Escher can be found at https://escher.readthedocs.io/en/latest/index.html

## Map Animation

Animations are relatively easy with an Escher map. Here's a simple example where flux predictions are swept across and range to update the map of *E. coli* core metabolism.


In [36]:
builder = Builder(
    height=600, 
    map_name='e_coli_core.Core metabolism', 
    model_name='e_coli_core',
)
builder

Builder(height=600)

In [39]:
from time import sleep
timestep = 0.5
lim = [0, 0.5]
for lb in range(-30,1,1):
    model.reactions.EX_o2_e.lower_bound = lb
    solution = model.optimize()
    builder.reaction_data = solution.fluxes
    sleep(timestep)

In this simulation, the o2 level starts at -30 mmol/gDW-hr which puts the organism in an aerobic state. As the o2 level drops to 0 mmol/gDW-hr, the organism passes through an anoxic state on it's way to an anaerobic state

## Saving Maps as HTML

The Escher Builder can also be saved as a standalone HTML file, which you can view by opening in a browser. Just provide a filepath, and the map will be bundled along with all the current options.


In [38]:
builder.save_html('example_map.html')

## References

1. https://cobrapy.readthedocs.io/en/latest/simulating.html#
2. https://escher.readthedocs.io/en/latest/index.html